# 画像の歪み補正

opencvのインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q opencv-python-headless

In [ ]:
%run setup_demo_env.py

In [ ]:
import cv2
import kachaka_api
import numpy as np
from IPython.display import Image, display

In [ ]:
client = kachaka_api.KachakaApiClient()

In [ ]:
image = client.get_front_camera_ros_compressed_image()
camera_info = client.get_front_camera_ros_camera_info()

In [ ]:
cv_image = cv2.imdecode(np.frombuffer(image.data, dtype=np.uint8), flags=1)

mtx = np.array(camera_info.K, dtype=float).reshape(3, 3)
dist = np.array(camera_info.D)
height = camera_info.height
width = camera_info.width
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(
    mtx, dist, (width, height), 0, (width, height)
)
map_x, map_y = cv2.initUndistortRectifyMap(
    mtx, dist, None, new_camera_mtx, (width, height), 5
)

undistorted_image = cv2.remap(cv_image, map_x, map_y, cv2.INTER_LINEAR)

In [ ]:
_, ret = cv2.imencode(".png", cv_image)
display(Image(data=ret, format="png"))

In [ ]:
_, ret = cv2.imencode(".png", undistorted_image)
display(Image(data=ret, format="png"))